In [1]:
def get_key():
  with open('20250723_OpenAI_API.key', 'r', encoding='utf-8') as file:
    return file.readline().rstrip()

In [2]:
!pip install langchain_openai langchain_community langchain_chroma pypdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.6/70.6 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 56.4 MB/s eta 0:00

In [3]:
import os
import requests
from typing import List
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever


In [4]:
os.environ['OPENAI_API_KEY'] = get_key()


In [5]:
# 분석할 PDF 파일을 웹에서 다운로드.
url = "https://github.com/llama-index-tutorial/llama-index-tutorial/raw/main/ch07/2023_%EB%B6%81%ED%95%9C%EC%9D%B8%EA%B6%8C%EB%B3%B4%EA%B3%A0%EC%84%9C.pdf"
filename = "2023_북한인권보고서.pdf"

response = requests.get(url)
with open(filename, "wb") as f:
    f.write(response.content)

print(f"{filename} 다운로드 완료")


2023_북한인권보고서.pdf 다운로드 완료


In [6]:
# LangChain의 LLM과 임베딩 모델 설정
llm = ChatOpenAI(model="gpt-4.1", temperature=0.2)  # GPT-4o를 언어 모델로 사용
embed_model = OpenAIEmbeddings(model="text-embedding-3-large")  # 임베딩 모델 사용

# 문서 분할 설정
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,  # 문서를 300자 단위로 분할
    chunk_overlap=100,  # 문맥 유지를 위해 청크 간 100자 중복
)

# PDF 문서를 읽고 벡터 인덱스 생성
loader = PyPDFLoader("2023_북한인권보고서.pdf")  # PDF 문서 로더
documents = loader.load()  # 문서에서 텍스트 추출
chunks = text_splitter.split_documents(documents)  # 문서 분할
vector_store = Chroma.from_documents(chunks, embed_model)  # 추출된 텍스트로 벡터 인덱스 생성


In [7]:
# 기본 검색 엔진 (리랭킹 없음)
basic_retriever = vector_store.as_retriever(search_kwargs={"k": 4})

# Reranker 설정
cross_encoder = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-v2-m3")
reranker = CrossEncoderReranker(model=cross_encoder, top_n=2)

# 리랭킹이 포함된 검색 엔진
rerank_retriever = ContextualCompressionRetriever(
    base_compressor=reranker,
    base_retriever=basic_retriever
)

# 최종 답변 생성 함수
def generate_answer(query: str, documents: List[Document]) -> str:
    context = "\n\n".join([doc.page_content for doc in documents])

    prompt = f"""다음 검색 결과를 바탕으로 질문에 답변해주세요.
    검색 결과의 정보를 최대한 사용하고, 없는 정보는 답변하지 마세요.

    검색 결과:
    {context}

    질문: {query}

    답변:"""

    response = llm.invoke(prompt)
    return response.content


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [8]:
# 쿼리 실행
query = "19년 말 평양시 소재 기업소에서 달마다 배급받은 음식"

print("=== 기본 검색 엔진 검색 결과 ===")
basic_documents = basic_retriever.invoke(query)
basic_response = generate_answer(query, basic_documents)

print(f"\n질문: {query}")
print(f"답변: {basic_response}")
print("\n검색된 문서:")
for i, doc in enumerate(basic_documents):
    print(f"\n검색 문서 {i+1}:")
    print(doc.page_content)
    print("---")


=== 기본 검색 엔진 검색 결과 ===

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 말 평양시 소재 기업소에서 일하던 노동자는 매월 쌀 6㎏, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리를 배급받았다는 증언이 있습니다.

검색된 문서:

검색 문서 1:
화 또는 쌀이나 기름 등 현물로 지급하였다고 한다. 2019년 평양
의 외화벌이 사업소에서는 보수 50달러를 월 2회로 나누어 현금으
로 지급하였다고 하는 사례가 있었고, 평양 외화벌이 식당에서는 매
---

검색 문서 2:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
---

검색 문서 3:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가
---

검색 문서 4:
한 달을 생활하기에 부족한 금액이었다고 하였다. 2018년 양강도의 
무역사업소에서는 1년치 노동 보수와 배급을 한 번에 지급하였다고 
하는데, 지급된 금액은 노동자 1명에게 1,800위안으로 약 300만원 
정도였다고 하였다.

In [9]:
print("\n\n=== 리랭킹 후 검색 결과 ===")
rerank_documents = rerank_retriever.invoke(query)
rerank_response = generate_answer(query, rerank_documents)

print(f"\n질문: {query}")
print(f"답변: {rerank_response}")
print("\n검색된 문서:")
for i, doc in enumerate(rerank_documents):
    print(f"\n검색 문서 {i+1}:")
    print(doc.page_content)
    print("---")




=== 리랭킹 후 검색 결과 ===

질문: 19년 말 평양시 소재 기업소에서 달마다 배급받은 음식
답변: 2019년 말 평양시 소재 기업소에서 일하는 노동자는 매월 쌀 6㎏, 기름 5ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 등을 배급받았습니다.

검색된 문서:

검색 문서 1:
파악되었다. 따라서 기관·기업소의 상황에 따라 식량배급량, 주기, 
곡식종류에 상당한 차이가 있는 것으로 나타났다. 외화벌이 기관 등
에는 식량배급이 원활하게 이뤄지고 있었다는 증언이 수집되었다. 
2019년 평양시에서 기업소 운전원으로 일하였던 노동자는 매월 쌀·
설탕·기름·야채·돼지고기 등을 배급받아 식량이 부족하지 않았다는 
증언과 2019년 중앙당 산하의 기업소에서 매월 쌀 6㎏ 정도, 기름 5
ℓ, 설탕 2㎏, 맛내기 2봉지, 돼지고기 2㎏, 닭고기 1마리 정도 받았
---

검색 문서 2:
가배급을 선택하고, 잘사는 기업소들은 기업소 자체 배급을 선택합
니 다. 세대주가 직장에 다닐 경우 세대주만 직장에서 배급을 받고 
가족들은 국가배급소에서 배급을 받습니다. 평양시와 자강도는 대
체로 다 줬는데 다른 지역은 배급이 잘 안되고 배급제가 없어졌다는 
소리를 들었습니다. ”
국가배급의 주기, 양, 곡물의 종류 등에서 평양시와 지방의 차이
가 크게 나고 있었다. 식량배급이 비교적 원활하게 작동하는 지역은 
평양시로 보이는데, 2017년 어머니가 지역배급 대상자로 배급표가
---
